In [1]:
print("ok")

ok


In [2]:
%pwd

'd:\\PROJECTS\\Gen AI Project\\Medical-Chatbot-Gen-AI\\research'

In [3]:
import os
os.chdir("../")

In [5]:
%pwd

'd:\\PROJECTS\\Gen AI Project\\Medical-Chatbot-Gen-AI'

In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract data from the PDF File
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [8]:
extracted_data = load_pdf("Data/")


In [ ]:
#extracted_data


In [9]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [10]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 39994


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

In [12]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [13]:
embeddings = download_hugging_face_embeddings()

C:\Users\kolde\AppData\Local\Temp\ipykernel_5948\1769223301.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\Softwares\envs\MediMate\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [42]:
#query_result

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [20]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medimate"


pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region= "us-east-1"
    )
)

In [21]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [23]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)

In [24]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your Pinecode index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [27]:
retrieved_docs = retriever.invoke("What is Acne")

In [28]:
retrieved_docs

[Document(id='9c54785e-d15a-41f8-8e0d-4ac22311ea98', metadata={'page': 55.0, 'page_label': '26', 'source': 'Data\\Medical_Book.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='dc98a48a-5094-4cbc-a844-b30f4d80043c', metadata={'page': 55.0, 'page_label': '26', 'source': 'Data\\Medical_Book.pdf'}, page_content='Researchers, Inc. Reproduced by permission.)\n26 GALE ENCYCLOPEDIA OF MEDICINE\nAcne'),
 Document(id='4f709515-f8f6-4381-a3e5-7479d706df37', metadata={'page': 269.0, 'page_label': '240', 'source': 'Data\\Medical_Book.pdf'}, page_content='forms ofacne.\nPurpose\nDifferent types of antiacne drugs are used for\ndifferent purposes. For example, lotions, soaps, gels,\nand creams containing benzoyl peroxide or tretinoin\nmay be used to clear up mild to moderately severe\nacne. Isotretinoin (Accutane) is prescribed only for\nvery severe, disfiguring acne.\nAcne is a skin condition that occurs when pores or\nhair fol